# Model 4 v3: Character-CNN + BiLSTM-CRF (No Vocabulary Filtering)

**Experiment: MIN_WORD_FREQ = 1 (use ALL words)**

This version tests whether including ALL words in vocabulary improves performance.

**Difference from v2:**
- ✅ **MIN_WORD_FREQ = 1**: Include every word (no filtering)
- Expected vocab size: ~40,000+ words (vs 36,790 in v2)

**Hypothesis:**
- ✅ Pro: No information loss, every word gets its own embedding
- ⚠️ Con: Larger model, potential overfitting to rare words

**All other improvements from v2 remain:**
1. Higher Dropout (0.6)
2. Dropout on Embeddings
3. Learning Rate Scheduler
4. Increased Patience (7)
5. GloVe 300d
6. Gradient Clipping (1.0)

**Expected Result:**
- If v3 ≈ v2: CharCNN handles rare words well, filtering doesn't matter
- If v3 > v2: Rare word information is valuable
- If v3 < v2: Rare words cause overfitting

## 1. Setup and Imports

In [1]:
# Install required packages using the notebook's Python interpreter
import sys
print(f"Jupyter kernel Python: {sys.executable}")

# Install packages to the correct Python environment
!{sys.executable} -m pip uninstall -y torchcrf
!{sys.executable} -m pip install torch pytorch-crf gensim tqdm

print("\n✅ Packages installed! Please RESTART THE KERNEL before continuing.")
print("   Kernel → Restart Kernel (or Ctrl+Shift+P → 'Restart Kernel')")

Jupyter kernel Python: /usr/local/bin/python3

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

✅ Packages installed! Please RESTART THE KERNEL before continuing.
   Kernel → Restart Kernel (or Ctrl+Shift+P → 'Restart Kernel')


In [2]:
import json
import numpy as np
import pickle
import time
from collections import Counter
import os

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# CRF
from torchcrf import CRF

# Embeddings
import gensim.downloader as api

# Progress bar
from tqdm import tqdm

# Our evaluation utilities
from utils import print_evaluation_report, evaluate_entity_spans

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

print("Imports successful!")

Using device: cpu
Imports successful!


## 2. Load Data

In [3]:
def load_jsonl(file_path):
    """Load JSONL file"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

# Load data
train_data = load_jsonl('train_split.jsonl')
val_data = load_jsonl('val_split.jsonl')

print(f"Training samples: {len(train_data):,}")
print(f"Validation samples: {len(val_data):,}")

# Extract tokens and tags
train_tokens = [sample['tokens'] for sample in train_data]
train_tags = [sample['ner_tags'] for sample in train_data]

val_tokens = [sample['tokens'] for sample in val_data]
val_tags = [sample['ner_tags'] for sample in val_data]

Training samples: 90,320
Validation samples: 10,036


## 3. Build Vocabularies

**🔬 EXPERIMENT: MIN_WORD_FREQ = 1 (include ALL words)**

In [4]:
# Build word vocabulary
word_counts = Counter()
for tokens in train_tokens:
    word_counts.update(tokens)

# 🔬 EXPERIMENT: Use ALL words (no filtering)
MIN_WORD_FREQ = 1  # Changed from 2 to 1!
word2idx = {'<PAD>': 0, '<UNK>': 1}
for word, count in word_counts.items():
    if count >= MIN_WORD_FREQ:
        word2idx[word] = len(word2idx)

idx2word = {idx: word for word, idx in word2idx.items()}
vocab_size = len(word2idx)

print(f"🔬 MIN_WORD_FREQ: {MIN_WORD_FREQ} (using ALL words)")
print(f"Word vocabulary size: {vocab_size:,}")
print(f"   v2 had: 36,790 words (with MIN_WORD_FREQ=2)")
print(f"   v3 has: {vocab_size:,} words (with MIN_WORD_FREQ=1)")
print(f"   Difference: +{vocab_size - 36790:,} words")

🔬 MIN_WORD_FREQ: 1 (using ALL words)
Word vocabulary size: 107,643
   v2 had: 36,790 words (with MIN_WORD_FREQ=2)
   v3 has: 107,643 words (with MIN_WORD_FREQ=1)
   Difference: +70,853 words


In [5]:
# Build character vocabulary
char2idx = {'<PAD>': 0, '<UNK>': 1}

chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 .,!?:;\'"()-[]{}@#$%^&*+=/<>\\|`~_'
for char in chars:
    if char not in char2idx:
        char2idx[char] = len(char2idx)

idx2char = {idx: char for char, idx in char2idx.items()}
char_vocab_size = len(char2idx)

print(f"Character vocabulary size: {char_vocab_size}")

Character vocabulary size: 97


In [6]:
# Build tag vocabulary
tag2idx = {}
for tags in train_tags:
    for tag in tags:
        if tag not in tag2idx:
            tag2idx[tag] = len(tag2idx)

idx2tag = {idx: tag for tag, idx in tag2idx.items()}
num_tags = len(tag2idx)

print(f"Number of NER tags: {num_tags}")
print(f"Tags: {list(tag2idx.keys())}")

Number of NER tags: 15
Tags: ['O', 'B-ORG', 'I-ORG', 'B-Facility', 'I-Facility', 'B-OtherPER', 'I-OtherPER', 'B-Politician', 'I-Politician', 'B-HumanSettlement', 'I-HumanSettlement', 'B-Artist', 'I-Artist', 'B-PublicCorp', 'I-PublicCorp']


## 4. Load Pre-trained Word Embeddings (GloVe 300d)

In [7]:
print("Downloading GloVe 300d embeddings (this may take ~5-10 minutes)...")
print("Using glove-wiki-gigaword-300 (300-dimensional, 400K vocabulary)")

# Download GloVe 300d embeddings
glove_model = api.load('glove-wiki-gigaword-300')

EMBEDDING_DIM = 300
print(f"\nGloVe 300d embeddings loaded!")
print(f"Embedding dimension: {EMBEDDING_DIM}")

Using glove-wiki-gigaword-300 (300-dimensional, 400K vocabulary)

GloVe 300d embeddings loaded!
Embedding dimension: 300


In [8]:
# Create embedding matrix
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

# Initialize with GloVe vectors
found = 0
for word, idx in word2idx.items():
    if word in ['<PAD>', '<UNK>']:
        continue
    
    try:
        embedding_matrix[idx] = glove_model[word.lower()]
        found += 1
    except KeyError:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))

# Special tokens
embedding_matrix[word2idx['<PAD>']] = np.zeros(EMBEDDING_DIM)
embedding_matrix[word2idx['<UNK>']] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))

print(f"Words found in GloVe: {found:,} / {vocab_size:,} ({found/vocab_size*100:.1f}%)")
print(f"Words initialized randomly: {vocab_size - found:,}")
print(f"\n📊 Comparison with v2:")
print(f"   v2: 33,849 / 36,790 (92.0%) found in GloVe")
print(f"   v3: {found:,} / {vocab_size:,} ({found/vocab_size*100:.1f}%) found in GloVe")

Words found in GloVe: 66,859 / 107,643 (62.1%)
Words initialized randomly: 40,784

📊 Comparison with v2:
   v2: 33,849 / 36,790 (92.0%) found in GloVe
   v3: 66,859 / 107,643 (62.1%) found in GloVe


## 5. Dataset Class

In [9]:
MAX_CHAR_LEN = 20

class NERDataset(Dataset):
    def __init__(self, tokens_list, tags_list, word2idx, char2idx, tag2idx):
        self.tokens_list = tokens_list
        self.tags_list = tags_list
        self.word2idx = word2idx
        self.char2idx = char2idx
        self.tag2idx = tag2idx
    
    def __len__(self):
        return len(self.tokens_list)
    
    def __getitem__(self, idx):
        tokens = self.tokens_list[idx]
        tags = self.tags_list[idx]
        
        word_ids = [self.word2idx.get(token, self.word2idx['<UNK>']) for token in tokens]
        
        char_ids = []
        for token in tokens:
            chars = [self.char2idx.get(c, self.char2idx['<UNK>']) for c in token[:MAX_CHAR_LEN]]
            if len(chars) < MAX_CHAR_LEN:
                chars += [self.char2idx['<PAD>']] * (MAX_CHAR_LEN - len(chars))
            char_ids.append(chars)
        
        tag_ids = [self.tag2idx[tag] for tag in tags]
        
        return {
            'word_ids': torch.LongTensor(word_ids),
            'char_ids': torch.LongTensor(char_ids),
            'tag_ids': torch.LongTensor(tag_ids),
            'length': len(tokens)
        }

train_dataset = NERDataset(train_tokens, train_tags, word2idx, char2idx, tag2idx)
val_dataset = NERDataset(val_tokens, val_tags, word2idx, char2idx, tag2idx)

print(f"Train dataset: {len(train_dataset)} samples")
print(f"Val dataset: {len(val_dataset)} samples")

Train dataset: 90320 samples
Val dataset: 10036 samples


In [10]:
# Collate function
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: x['length'], reverse=True)
    
    word_ids = [item['word_ids'] for item in batch]
    char_ids = [item['char_ids'] for item in batch]
    tag_ids = [item['tag_ids'] for item in batch]
    lengths = [item['length'] for item in batch]
    
    word_ids_padded = pad_sequence(word_ids, batch_first=True, padding_value=word2idx['<PAD>'])
    tag_ids_padded = pad_sequence(tag_ids, batch_first=True, padding_value=tag2idx['O'])
    
    max_len = max(1, word_ids_padded.size(1))
    batch_size = len(batch)
    
    char_ids_padded = torch.full(
        (batch_size, max_len, MAX_CHAR_LEN),
        fill_value=char2idx['<PAD>'],
        dtype=torch.long
    )
    
    for i, chars in enumerate(char_ids):
        seq_len = chars.size(0)
        if seq_len > 0:
            char_ids_padded[i, :seq_len, :] = chars
    
    mask = torch.zeros((batch_size, max_len), dtype=torch.bool)
    for i, length in enumerate(lengths):
        if length > 0:
            mask[i, :length] = True
    
    return {
        'word_ids': word_ids_padded,
        'char_ids': char_ids_padded,
        'tag_ids': tag_ids_padded,
        'lengths': lengths,
        'mask': mask
    }

BATCH_SIZE = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

Train batches: 2823
Val batches: 314


## 6. Model Architecture (Same as v2)

In [11]:
class CharCNN(nn.Module):
    def __init__(self, char_vocab_size, char_emb_dim, char_hidden_dim, max_char_len, dropout=0.6):
        super().__init__()
        self.char_embedding = nn.Embedding(char_vocab_size, char_emb_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout)
        
        self.conv = nn.Conv1d(
            in_channels=char_emb_dim,
            out_channels=char_hidden_dim,
            kernel_size=3,
            padding=1
        )
        self.relu = nn.ReLU()
    
    def forward(self, char_ids):
        batch_size, seq_len, max_char_len = char_ids.size()
        
        char_ids = char_ids.view(-1, max_char_len)
        char_embeds = self.char_embedding(char_ids)
        char_embeds = self.dropout(char_embeds)
        char_embeds = char_embeds.transpose(1, 2)
        
        char_conv = self.relu(self.conv(char_embeds))
        char_features = torch.max(char_conv, dim=2)[0]
        char_features = char_features.view(batch_size, seq_len, -1)
        
        return char_features


class BiLSTM_CRF_v3(nn.Module):
    """Same as v2, but with larger vocabulary"""
    def __init__(self, vocab_size, char_vocab_size, embedding_dim, char_emb_dim,
                 char_hidden_dim, lstm_hidden_dim, num_tags, dropout=0.6, embedding_matrix=None):
        super().__init__()
        
        # Word embeddings
        self.word_embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if embedding_matrix is not None:
            self.word_embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
            self.word_embedding.weight.requires_grad = True
        
        self.embedding_dropout = nn.Dropout(dropout)
        
        # Character CNN
        self.char_cnn = CharCNN(char_vocab_size, char_emb_dim, char_hidden_dim, MAX_CHAR_LEN, dropout=dropout)
        
        # BiLSTM
        self.lstm = nn.LSTM(
            input_size=embedding_dim + char_hidden_dim,
            hidden_size=lstm_hidden_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )
        
        self.lstm_dropout = nn.Dropout(dropout)
        
        # Linear layer
        self.fc = nn.Linear(lstm_hidden_dim * 2, num_tags)
        
        # CRF
        self.crf = CRF(num_tags, batch_first=True)
    
    def forward(self, word_ids, char_ids, tags=None, mask=None):
        # Word embeddings with dropout
        word_embeds = self.word_embedding(word_ids)
        word_embeds = self.embedding_dropout(word_embeds)
        
        # Character features
        char_features = self.char_cnn(char_ids)
        
        # Concatenate
        combined = torch.cat([word_embeds, char_features], dim=-1)
        
        # BiLSTM with dropout
        lstm_out, _ = self.lstm(combined)
        lstm_out = self.lstm_dropout(lstm_out)
        
        # Emission scores
        emissions = self.fc(lstm_out)
        
        if tags is not None:
            loss = -self.crf(emissions, tags, mask=mask, reduction='mean')
            return loss
        else:
            predictions = self.crf.decode(emissions, mask=mask)
            return predictions

print("✅ Model architecture defined (v3 with larger vocabulary)!")

✅ Model architecture defined (v3 with larger vocabulary)!


## 7. Initialize Model

In [12]:
# Hyperparameters (same as v2)
CHAR_EMB_DIM = 25
CHAR_HIDDEN_DIM = 30
LSTM_HIDDEN_DIM = 256
DROPOUT = 0.6
LEARNING_RATE = 0.001
NUM_EPOCHS = 20

# Initialize model
model = BiLSTM_CRF_v3(
    vocab_size=vocab_size,
    char_vocab_size=char_vocab_size,
    embedding_dim=EMBEDDING_DIM,
    char_emb_dim=CHAR_EMB_DIM,
    char_hidden_dim=CHAR_HIDDEN_DIM,
    lstm_hidden_dim=LSTM_HIDDEN_DIM,
    num_tags=num_tags,
    dropout=DROPOUT,
    embedding_matrix=embedding_matrix
).to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=2, verbose=True
)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())

print(f"\n✅ Model initialized (v3)!")
print(f"Total parameters: {total_params:,}")
print(f"\n📊 Comparison with v2:")
print(f"   v2: 12,253,879 parameters (vocab=36,790)")
print(f"   v3: {total_params:,} parameters (vocab={vocab_size:,})")
print(f"   Difference: +{total_params - 12253879:,} parameters")


✅ Model initialized (v3)!
Total parameters: 33,509,779

📊 Comparison with v2:
   v2: 12,253,879 parameters (vocab=36,790)
   v3: 33,509,779 parameters (vocab=107,643)
   Difference: +21,255,900 parameters


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


## 8. Training Loop

In [13]:
def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    num_batches = 0
    
    pbar = tqdm(data_loader, desc="Training", leave=False)
    
    for batch in pbar:
        word_ids = batch['word_ids'].to(device)
        char_ids = batch['char_ids'].to(device)
        tag_ids = batch['tag_ids'].to(device)
        mask = batch['mask'].to(device)
        lengths = batch['lengths']
        
        # Filter empty sequences
        non_empty_indices = [i for i, length in enumerate(lengths) if length > 0]
        
        if len(non_empty_indices) == 0:
            continue
        
        if len(non_empty_indices) < len(lengths):
            word_ids = word_ids[non_empty_indices]
            char_ids = char_ids[non_empty_indices]
            tag_ids = tag_ids[non_empty_indices]
            mask = mask[non_empty_indices]
        
        optimizer.zero_grad()
        loss = model(word_ids, char_ids, tag_ids, mask)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    return total_loss / max(num_batches, 1)


def evaluate(model, data_loader, device):
    model.eval()
    all_predictions = []
    all_true_tags = []
    
    with torch.no_grad():
        pbar = tqdm(data_loader, desc="Evaluating", leave=False)
        
        for batch in pbar:
            word_ids = batch['word_ids'].to(device)
            char_ids = batch['char_ids'].to(device)
            tag_ids = batch['tag_ids'].to(device)
            mask = batch['mask'].to(device)
            lengths = batch['lengths']
            
            non_empty_indices = [i for i, length in enumerate(lengths) if length > 0]
            
            if len(non_empty_indices) > 0:
                word_ids_non_empty = word_ids[non_empty_indices]
                char_ids_non_empty = char_ids[non_empty_indices]
                mask_non_empty = mask[non_empty_indices]
                
                predictions_non_empty = model(word_ids_non_empty, char_ids_non_empty, mask=mask_non_empty)
            else:
                predictions_non_empty = []
            
            predictions = []
            non_empty_iter = iter(predictions_non_empty)
            for i in range(len(lengths)):
                if lengths[i] == 0:
                    predictions.append([])
                else:
                    predictions.append(next(non_empty_iter))
            
            for i, (pred, length) in enumerate(zip(predictions, lengths)):
                if length == 0:
                    pred_tags = []
                    true_tags = []
                else:
                    pred_tags = [idx2tag[idx] for idx in pred[:length]]
                    true_tags = [idx2tag[tag_ids[i][j].item()] for j in range(length)]
                
                all_predictions.append(pred_tags)
                all_true_tags.append(true_tags)
    
    return all_true_tags, all_predictions

print("Training functions defined!")

Training functions defined!


## 9. Train Model

In [14]:
print("Starting training (v3 with MIN_WORD_FREQ=1)...\n")
print("=" * 80)
print("🔬 Experiment: MIN_WORD_FREQ = 1 (ALL words)")
print(f"   Vocabulary size: {vocab_size:,} (vs 36,790 in v2)")
print(f"   Model parameters: {total_params:,} (vs 12,253,879 in v2)")
print("="  * 80 + "\n")

best_f1 = 0
patience = 7
patience_counter = 0

training_start = time.time()

for epoch in range(NUM_EPOCHS):
    epoch_start = time.time()
    
    # Train
    train_loss = train_epoch(model, train_loader, optimizer, device)
    
    # Evaluate
    val_true_tags, val_pred_tags = evaluate(model, val_loader, device)
    
    # Calculate F1
    results = evaluate_entity_spans(val_true_tags, val_pred_tags, val_tokens)
    val_f1 = results['f1']
    val_precision = results['precision']
    val_recall = results['recall']
    
    # Update learning rate scheduler
    scheduler.step(val_f1)
    
    epoch_time = time.time() - epoch_start
    current_lr = optimizer.param_groups[0]['lr']
    
    print(f"Epoch {epoch+1:2d}/{NUM_EPOCHS} | "
          f"Loss: {train_loss:.4f} | "
          f"Val P: {val_precision:.4f} R: {val_recall:.4f} F1: {val_f1:.4f} | "
          f"LR: {current_lr:.6f} | "
          f"Time: {epoch_time:.1f}s")
    
    # Early stopping
    if val_f1 > best_f1:
        best_f1 = val_f1
        patience_counter = 0
        os.makedirs('models', exist_ok=True)
        torch.save(model.state_dict(), 'models/bilstm_crf_v3_best.pt')
        print(f"  → New best F1! Model saved.")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping after {epoch+1} epochs (patience={patience})")
            break

training_time = time.time() - training_start

print("=" * 80)
print(f"\nTraining completed in {training_time:.1f}s ({training_time/60:.1f} minutes)")
print(f"Best validation F1: {best_f1:.4f}")
print(f"\n📊 Comparison with v2:")
print(f"   v2 (MIN_WORD_FREQ=2): 0.7594 F1")
print(f"   v3 (MIN_WORD_FREQ=1): {best_f1:.4f} F1")
print(f"   Difference: {(best_f1 - 0.7594)*100:+.2f}% absolute")
print(f"\n🔬 Conclusion:")
if best_f1 > 0.7594 + 0.001:
    print("   ✅ Using ALL words improves performance!")
elif best_f1 < 0.7594 - 0.001:
    print("   ❌ Filtering rare words is better (prevents overfitting)")
else:
    print("   ≈ No significant difference - CharCNN handles rare words well!")

Starting training (v3 with MIN_WORD_FREQ=1)...

🔬 Experiment: MIN_WORD_FREQ = 1 (ALL words)
   Vocabulary size: 107,643 (vs 36,790 in v2)
   Model parameters: 33,509,779 (vs 12,253,879 in v2)



Epoch  1/20 | Loss: 3.5427 | Val P: 0.7260 R: 0.6418 F1: 0.6813 | LR: 0.001000 | Time: 309.1s
  → New best F1! Model saved.


Epoch  2/20 | Loss: 1.8265 | Val P: 0.7513 R: 0.6660 F1: 0.7060 | LR: 0.001000 | Time: 303.8s
  → New best F1! Model saved.


Epoch  3/20 | Loss: 1.4933 | Val P: 0.7694 R: 0.6854 F1: 0.7250 | LR: 0.001000 | Time: 290.0s
  → New best F1! Model saved.


Epoch  4/20 | Loss: 1.3019 | Val P: 0.7714 R: 0.7029 F1: 0.7356 | LR: 0.001000 | Time: 282.6s
  → New best F1! Model saved.


...

Epoch  8/20 | Loss: 0.8750 | Val P: 0.7765 R: 0.7303 F1: 0.7527 | LR: 0.001000 | Time: 309.9s


Epoch  9/20 | Loss: 0.8073 | Val P: 0.7816 R: 0.7306 F1: 0.7552 | LR: 0.001000 | Time: 304.9s
  → New best F1! Model saved.


Epoch 10/20 | Loss: 0.7446 | Val P: 0.7773 R: 0.7249 F1: 0.7502 | LR: 0.001000 | Time: 310.5s


Epoch 11/20 | Loss: 0.6929 | Val P: 0.7763 R: 0.7239 F1: 0.7492 | LR: 0.001000 | Time: 294.0s


KeyboardInterrupt: 

## 10. Load Best Model and Final Evaluation

In [15]:
# Load best model
model.load_state_dict(torch.load('models/bilstm_crf_v3_best.pt'))
model.eval()

print("Best model loaded!")

# Final evaluation
val_true_tags, val_pred_tags = evaluate(model, val_loader, device)

# Comprehensive report
print_evaluation_report(
    val_true_tags,
    val_pred_tags,
    val_tokens,
    model_name="Character-CNN + BiLSTM-CRF v3 (MIN_WORD_FREQ=1)"
)

Best model loaded!


ENTITY-SPAN LEVEL EVALUATION REPORT: Character-CNN + BiLSTM-CRF v3 (MIN_WORD_FREQ=1)

OVERALL METRICS:
  Precision: 0.7816
  Recall:    0.7306
  F1 Score:  0.7552

  True Positives:  8038
  False Positives: 2246
  False Negatives: 2964

--------------------------------------------------------------------------------
PER-ENTITY-TYPE METRICS:
--------------------------------------------------------------------------------
Entity Type          Precision    Recall       F1           Support   
--------------------------------------------------------------------------------
Artist               0.7313       0.8255       0.7756       2430      
Facility             0.7797       0.6547       0.7118       1173      
HumanSettlement      0.8978       0.9051       0.9014       2697      
ORG                  0.7871       0.6498       0.7119       1542      
OtherPER             0.6859       0.4977       0.5769       1527      
Politician           0.7374       0.6348       0.6822       1150     

## 11. Save Results

In [17]:
training_time = "55 minutes (early manual stop)"

In [18]:
# Save vocabularies
vocab_data = {
    'word2idx': word2idx,
    'char2idx': char2idx,
    'tag2idx': tag2idx,
    'idx2word': idx2word,
    'idx2char': idx2char,
    'idx2tag': idx2tag
}

with open('models/bilstm_crf_v3_vocab.pkl', 'wb') as f:
    pickle.dump(vocab_data, f)

print("Vocabularies saved!")

# Save results
final_results = evaluate_entity_spans(val_true_tags, val_pred_tags, val_tokens)

results_summary = {
    'model': 'Character-CNN + BiLSTM-CRF v3',
    'version': 'v3',
    'experiment': 'MIN_WORD_FREQ = 1 (all words)',
    'precision': final_results['precision'],
    'recall': final_results['recall'],
    'f1': final_results['f1'],
    'training_time': training_time,
    'num_epochs': epoch + 1,
    'hyperparameters': {
        'min_word_freq': MIN_WORD_FREQ,
        'vocab_size': vocab_size,
        'embedding_dim': EMBEDDING_DIM,
        'char_emb_dim': CHAR_EMB_DIM,
        'char_hidden_dim': CHAR_HIDDEN_DIM,
        'lstm_hidden_dim': LSTM_HIDDEN_DIM,
        'learning_rate': LEARNING_RATE,
        'batch_size': BATCH_SIZE,
        'dropout': DROPOUT,
        'patience': patience,
        'gradient_clipping': 1.0
    },
    'comparison': {
        'v2_f1': 0.7594,
        'v2_vocab_size': 36790,
        'v3_f1': final_results['f1'],
        'v3_vocab_size': vocab_size,
        'difference': final_results['f1'] - 0.7594
    }
}

with open('models/bilstm_crf_v3_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print("Results saved!")

Vocabularies saved!
Results saved!


## 12. Summary

### Experiment: MIN_WORD_FREQ = 1 vs 2

**Hypothesis Testing:**
- Does including ALL words (even those with frequency=1) improve NER performance?

**Results will show one of three outcomes:**

1. **v3 > v2**: Rare word information is valuable
   - Even words appearing once contain useful information
   - Proper nouns and rare entities benefit from dedicated embeddings

2. **v3 < v2**: Vocabulary filtering prevents overfitting
   - Frequency-1 words are often noise (typos, artifacts)
   - Smaller vocabulary generalizes better

3. **v3 ≈ v2**: No significant difference
   - CharCNN effectively handles rare/unknown words
   - Word-level embeddings less important when char features are strong

**Trade-offs:**
- Model size: v3 is larger (~1M more parameters)
- Training time: v3 may be slightly slower
- Memory: v3 requires more GPU memory

Check the results above to see which approach works better for your dataset!